In [ ]:
import math
import glob2
import numpy as np
import cv2
import pickle
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from shapely.geometry import Polygon
import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import *
from IPython.display import HTML
%matplotlib inline

class WindowTiler:

### Paramters #################################################################
    MASK_REGION = ( (750,600), (750,400), (1280,400), (1280,600) )
    TOP_SIZE = 70
    BOTTOM_SIZE = 150  
    OVERLAP = 0.92

### Function Set ##############################################################
    
### Initialization, compute polygon, roi, left=start_point, right=stop_point, top=start_size, bottom=stop_size
    def __init__(self, image):
        self.image = image
        self.poly = Polygon(self.MASK_REGION)
        roi = np.array(self.MASK_REGION)
        self.roi = roi
        self.left = np.amin(roi, axis=0)[0]
        self.right = np.amax(roi, axis=0)[0]
        self.top = np.amin(roi, axis=0)[1]
        self.bottom = np.amax(roi, axis=0)[1]

### Define a tiler function to generate proportional steps via vertical and horizontal iterator
    def tiler(self, start_point, stop_point, start_size, stop_size):
        i = start_point + start_size / 2
        size = start_size
        size_slope = float(stop_size - start_size) / float(stop_point - start_point)
        result = []
        while i < stop_point - stop_size / 2:
            result.append((int(i),int(size)))
            size = int(start_size + (i - start_point) * size_slope)
            i = int(i + size * (1 - self.OVERLAP))
        return result

### Generate filter (windows)
    def generate_windows(self):
    ### Compute regions
        regions = []
        vertical = self.tiler(self.top, self.bottom, self.TOP_SIZE, self.BOTTOM_SIZE)  # 纵向计算
        for v in vertical:
            horizontal = self.tiler(self.left, self.right, v[1], v[1])                #横向计算
            for h in horizontal:
                center = (h[0], v[0])
                size = (h[1], v[1])
                regions.append((center, size))
    ### Compute roi_area and fill polygon
        if len(self.image.shape) > 2:
            channel_count = self.image.shape[2]  
            ignore_mask_color = (255,) * channel_count
        else:
            ignore_mask_color = 255
        mask = np.zeros_like(self.image) 
        cv2.fillPoly(mask, [self.roi], ignore_mask_color)
        roi_image = cv2.bitwise_and(self.image, mask)
    ### Generate windows
        windows = []
        for r in regions:
            center = r[0]
            size = r[1]
            left = int(center[1] - size[1]/2)
            right = int(center[1] + size[1]/2)
            top = int(center[0] - size[0]/2)
            bottom = int(center[0] + size[0]/2)
            poly = ((bottom,left), (top,left), (top,right), (bottom,right))
            poly = Polygon(poly)
            intersects = poly.intersects(self.poly)
            if intersects:
                sub_image = roi_image[left:right,top:bottom]
                windows.append((center, size, sub_image))
        return windows

#------------------------------------------------------------------------------

class HeatMap:
    
### Initialization
    def __init__(self, image):
        self.image = np.array(image)
        self.height = self.image.shape[0]
        self.width = self.image.shape[1]

### 暂时并没什么用
    def mono_overlay(self, points, radius, intensity):    # intensity从外部引入
        k_size = 3 * radius
        kernel = np.ones((k_size, k_size), np.float32) / (k_size**2)
        mono_overlay = np.zeros((self.height, self.width), np.uint8)
        for p in points:
            p = (int(p[0]), int(p[1]))
            ol = np.zeros((self.height, self.width), np.uint8)
            cv2.circle(ol, p, radius, intensity, -1)
            ol = cv2.filter2D(ol, -1, kernel)
            cv2.addWeighted(ol, 1.0, overlay, 1.0, 0, color_overlay)
        return mono_overlay
    
### 暂时并没什么用
    def color_overlay(self, points, radius, color):    # color从外部引入
        k_size = 3 * radius
        kernel = np.ones((k_size, k_size), np.float32) / (k_size**2)
        color_overlay = np.zeros((self.height, self.width, 3), np.uint8)
        for p in points:
            p = (int(p[0]), int(p[1]))
            ol = np.zeros((self.height, self.width, 3), np.uint8)
            cv2.circle(ol, p, radius, color, -1)
            ol = cv2.filter2D(ol, -1, kernel)
            cv2.addWeighted(ol, 1.0, overlay, 1.0, 0, color_overlay)
        return color_overlay

#------------------------------------------------------------------------------

class processing:

### Paramters #################################################################

    PICKLE_FILE = 'classified_data.p'

### Function Set ##############################################################	

### Load classified data    
    def load_classifier(self):
        self.classifier = pickle.load(open(self.PICKLE_FILE, 'rb'))

### Load image
    def load_file(self, filename):
        self.image = mpimg.imread(filename)
        
### sort by descending order of array, swap data
    def sort(self, data):
        return (data[1], data[0])

### 用于car_points_heatmap和intensity_heatmap
    def car_points(self):
        wt = WindowTiler(self.image)
        windows = wt.generate_windows()
        points = []
        for w in windows:
            center = self.sort(w[0])
            bounds = w[1]
            ll = (int(center[0] - bounds[0] / 2), int(center[1] - bounds[1] / 2))
            ur = (int(center[0] + bounds[0] / 2), int(center[1] + bounds[1] / 2))
            corners = (ll, ur)
            image = w[2]
            is_car = self.classifier.classify_new(image)
            if is_car[2] == 1.0:
                points.append((center, corners))
        return points

### 暂时并没什么用
    def get_region_centers(self, regions):
        return [self.sort(r[0]) for r in regions]

### 暂时并没什么用
### Generate a mono overlay
    def intensity_heatmap(self, radius, intensity):
        heat_map = HeatMap(self.image)
        points = [self.sort(r[0]) for r in self.car_points()]
        mono_overlay = heat_map.mono_overlay(points, radius, intensity)
        return mono_overlay

### Generate a clolor overlay
    def car_points_heatmap(self, radius, color):
        heat_map = HeatMap(self.image)
        points = [self.sort(r[0]) for r in self.car_points()]
        color_overlay = heat_map.color_overlay(points, radius, color)
        return color_overlay
    
### Attach a overlay of head map
    def overlay(self, image, alpha):
        cv2.addWeighted(image, alpha, self.image, 1.0, 0, self.image)
        return self.image

#------------------------------------------------------------------------------
p = processing()
p.load_classifier()
p.load_file('test_images/test4.jpg')
cph = p.car_points_heatmap(20, (100,100,0))
plt.imshow(p.overlay(cph, alpha=1.0))

In [17]:
class Classifier:

### Paramters #################################################################

### combined_spatial()
    SPATIAL_SIZE = (32, 32)
    HLS_BINS = 512 
    HLS_BIN_RANGE = (0,255)
    HOG_ORIENTATIONS = 9
    HOG_PIXELS_PER_CELL = (8,8)
    HOG_CELLS_PER_BLOCK = (2,2)
### 用于load()
    IMAGE_SIZE = (64, 64)
    TEST_FRACTION = 0.2
    RANDOM_STATE = 12345
### train()
    SVC_KERNEL = 'rbf'
    SVC_C_VALUE = 2.0
    DT_MIN_SAMPLES_SPLIT = 40
### store(), define path 'classified_data.p' to save classified data
    PICKLE_FILE = 'classified_data.p'

### Function Set ##############################################################

### Initialization, vehicle=true, non-vehicle=false
    def __init__(self, true_path, false_path):
        self.true_path = true_path
        self.false_path = false_path

### Combine spatial function for load data
    def combined_spatial(self, image):
    ### compute spatial of image
        spatial = cv2.resize(image, self.SPATIAL_SIZE).ravel()
    ### compute spatial with hls
        hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        h_hist = np.histogram(hls[:,:,0], bins=self.HLS_BINS, range=self.HLS_BIN_RANGE)
        s_hist = np.histogram(hls[:,:,2], bins=self.HLS_BINS, range=self.HLS_BIN_RANGE)
        hls = np.concatenate((h_hist[0], s_hist[0]))
    ### compute spatial with hog
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        hogs = hog(gray, orientations=self.HOG_ORIENTATIONS, pixels_per_cell=self.HOG_PIXELS_PER_CELL, cells_per_block=self.HOG_CELLS_PER_BLOCK, transform_sqrt=True, visualise=False, feature_vector=True)
        combined_spatial = np.concatenate((spatial, hls, hogs))
        return combined_spatial

### Load and split data
    def load(self):
    ### Define the path of vehicle folder and non-vehicle folder
        true_paths = glob2.glob(self.true_path)
        false_paths = glob2.glob(self.false_path)
    ### Define features set and labels set for load images and store data
        features = []
        labels = []
    ### Load vehicle images and store features and labels, vehicle=1.0
        for filename in true_paths:
            image = cv2.resize(mpimg.imread(filename), self.IMAGE_SIZE)
            hls_hog = self.combined_spatial(image)
            features.append(hls_hog)
            labels.append(1.0)
    ### Load non-vehicle images and store features and labels, non-vehicle=0.0
        for filename in false_paths:
            image = cv2.resize(mpimg.imread(filename), self.IMAGE_SIZE)
            hls_hog = self.combined_spatial(image)
            features.append(hls_hog)
            labels.append(0.0)
    ### Normalize features
        features=np.array(features, np.float32)
        self.scaler = StandardScaler().fit(features)
        features = self.scaler.transform(features)
    ### Split to train features, train labels, test features, test labels, store train labels and test labels as numpy array
        self.train_features, self.test_features, self.train_labels, self.test_labels = train_test_split(features, labels, test_size=self.TEST_FRACTION, random_state=self.RANDOM_STATE)
        self.train_labels = np.array(self.train_labels, np.float32)
        self.test_labels = np.array(self.test_labels, np.float32)
        return self
    
    def combined_prediction(self, list_svc, list_dt):
        prediction = []
        for index, val_svc in enumerate(list_svc):
            val_dt = list_dt[index]
            if val_svc == 1.0 or val_dt == 1.0:
                prediction.append(1.0)
            else:
                prediction.append(0.0)
        return prediction
    
    def classify_new(self, image):
        image = cv2.resize(image, self.IMAGE_SIZE)
        hls_hog = self.combined_spatial(image)
        features=np.array(hls_hog, np.float32)
        self.scaler = StandardScaler().fit(features)
        features = self.scaler.transform(features)
        self.svc_classifier = SVC(kernel=self.SVC_KERNEL,C=self.SVC_C_VALUE)
        self.svc_classifier.fit(self.train_features, self.train_labels)
        svc_prediction = self.svc_classifier.predict(features)
        self.dt_classifier = DecisionTreeClassifier(min_samples_split=self.DT_MIN_SAMPLES_SPLIT)
        self.dt_classifier.fit(self.train_features, self.train_labels)
        dt_prediction = self.dt_classifier.predict(features)
        composite_prediction = self.combined_prediction(svc_prediction, dt_prediction)
        return np.array([svc_prediction, dt_prediction, composite_prediction])